In [113]:
import os
import json
import pandas as pd
import traceback

In [114]:
from langchain.chat_models import ChatOpenAI

In [115]:
from dotenv import load_dotenv


In [165]:
dotenv_path = os.path.abspath("../.env")

load_dotenv(dotenv_path=dotenv_path, override=True)
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm= ChatOpenAI(openai_api_key=KEY,
    model_name="gpt-4o",temperature=0.5)

In [120]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [121]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }, 
     "2": {
        "mcq": "multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }, 
     "3": {
        "mcq": "multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }, 
}

In [122]:
TEMPLATE = """

{text}

You are an expert MCQ creator.Given the above text,it is your job to create a quiz of {number} multiple choice question for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well. Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs
###RESPONE_JSON
{response_json}

"""


In [123]:
quiz_generating_prompt=PromptTemplate(input_variables=["text","number","subject","tone","response_json"],
                                      template=TEMPLATE
                                      )

In [124]:
quiz_chain=LLMChain(llm=llm,
                    prompt=quiz_generating_prompt,
                    output_key="quiz", 
                    verbose=True)

In [125]:
TEMPLATE2 = """
You are an expert grammarian and educational reviewer.Given the following MCQ quiz for {subject} students.Ypu need to evaluate  the complexity of  the question and give a complete analysis of the quiz.Onlye use at max 50 words for complexity.
if the quiz is not at per with the cognative and analytical of the student,update the quiz question which needs to be changed the tone that us perfectly fits the student ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [126]:
quiz_evaluating_prompt=PromptTemplate(input_variables=["quiz","subject"],
                                      template=TEMPLATE2)

In [127]:
review_chain=LLMChain(llm=llm,
                      prompt=quiz_evaluating_prompt,
                      output_key="review",
                      verbose=True)

In [128]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],
                                        input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"],
                                        verbose=True)

In [129]:
file_path=r"C:\Users\UPASHAK GAYEN\OneDrive\Desktop\mcqgen\data.txt"

In [130]:
with open(file_path, "r") as file:
    TEXT =file.read()

In [131]:
#serializing the Python Dictionary to a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [132]:
SUBJECT = "Machine Learning"
TONE="simple"

In [ ]:
with get_openai_callback() as cb:
        response = generate_evaluate_chain.invoke({
            "text": TEXT,
            "number": 3,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        })
        print("Response:", response)
        print("Type:", type(response))

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:


Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is provided feedback that's analogous to rewards, which it tries to maximise.[5]
Althoug

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert grammarian and educational reviewer.Given the following MCQ quiz for Machine Learning students.Ypu need to evaluate  the complexity of  the question and give a complete analysis of the quiz.Onlye use at max 50 words for complexity.
if the quiz is not at per with the cognative and analytical of the student,update the quiz question which needs to be changed the tone that us perfectly fits the student ability.
Quiz_MCQs:
```json
{
  "1": {
    "mcq": "What is the main goal of supervised learning?",
    "options": {
      "a": "To discover hidden patterns in data",
      "b": "To interact with a dynamic environment",
      "c": "To learn a general rule that maps inputs to outputs",
      "d": "To perform a certain goal like playing a game"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "Which learning paradigm involves the algorithm finding structure in its input without given labels?",
    "options": {
      "a": "Supervis

In [159]:
response

{'text': 'Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:\n\nSupervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.\nUnsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).\nReinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is provided feedback that\'s analogous to rewards, which it tries to maximise.[5]\nAlthough each algo

In [160]:
response.get("quiz")

'```json\n{\n  "1": {\n    "mcq": "What is the main goal of supervised learning?",\n    "options": {\n      "a": "To discover hidden patterns in data",\n      "b": "To interact with a dynamic environment",\n      "c": "To learn a general rule that maps inputs to outputs",\n      "d": "To perform a certain goal like playing a game"\n    },\n    "correct": "c"\n  },\n  "2": {\n    "mcq": "Which learning paradigm involves the algorithm finding structure in its input without given labels?",\n    "options": {\n      "a": "Supervised learning",\n      "b": "Unsupervised learning",\n      "c": "Reinforcement learning",\n      "d": "None of the above"\n    },\n    "correct": "b"\n  },\n  "3": {\n    "mcq": "In which learning approach does a program receive feedback similar to rewards?",\n    "options": {\n      "a": "Supervised learning",\n      "b": "Unsupervised learning",\n      "c": "Reinforcement learning",\n      "d": "None of the above"\n    },\n    "correct": "c"\n  }\n}\n```'

In [ ]:
import json
import re

quiz_raw = response.get('quiz')  

quiz_cleaned = re.sub(r"```json|```", "", quiz_raw).strip()

quiz_dict = json.loads(quiz_cleaned)

print(json.dumps(quiz_dict, indent=2))


{
  "1": {
    "mcq": "What is the main goal of supervised learning?",
    "options": {
      "a": "To discover hidden patterns in data",
      "b": "To interact with a dynamic environment",
      "c": "To learn a general rule that maps inputs to outputs",
      "d": "To perform a certain goal like playing a game"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "Which learning paradigm involves the algorithm finding structure in its input without given labels?",
    "options": {
      "a": "Supervised learning",
      "b": "Unsupervised learning",
      "c": "Reinforcement learning",
      "d": "None of the above"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "In which learning approach does a program receive feedback similar to rewards?",
    "options": {
      "a": "Supervised learning",
      "b": "Unsupervised learning",
      "c": "Reinforcement learning",
      "d": "None of the above"
    },
    "correct": "c"
  }
}


In [ ]:
data = []
for q_no, q_data in quiz_dict.items():
    mcq= q_data["mcq"]
    options = " | ".join([
        f"{key}: {value}" for key, value in q_data["options"].items()
    ])
    correct = q_data["correct"]
    data.append({"MCQ": mcq, "Options": options, "Correct Answer": correct  })

df = pd.DataFrame(data)
df

,MCQ,Options,Correct Answer
0,What is the main goal of supervised learning?,a: To discover hidden patterns in data | b: To...,c
1,Which learning paradigm involves the algorithm...,a: Supervised learning | b: Unsupervised learn...,b
2,In which learning approach does a program rece...,a: Supervised learning | b: Unsupervised learn...,c


In [164]:
df.to_csv("quiz.csv", index=False)